In [1]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)


TensorFlow version: 2.12.0
Keras version: 2.12.0


In [2]:
import os

In [3]:
from keras.preprocessing import image

In [4]:
categories = ['with_mask','without_mask']


In [5]:
for category in categories :
    path = os.path.join('train',category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        print(file)
        break

0-with-mask_jpg.rf.2dd114e4f143ba8bf221a0377529b7a5.jpg
0_jpg.rf.2bfbd6a3b285a3a25505d2c37cdcc11d.jpg


In [6]:
import cv2

In [7]:
for category in categories :
    path = os.path.join('train',category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        print(img.shape)
        break

(224, 224, 3)
(224, 224, 3)


In [8]:
data = []
for category in categories :
    path = os.path.join('train',category)
    label = categories.index(category)
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        data.append([img,label])

In [9]:
len(data)

841

In [10]:
import random
random.shuffle(data)

In [11]:
X = []
Y = []

for features,label in data:
    X.append(features)
    Y.append(label)

In [12]:
len(X)

841

In [13]:
len(Y)

841

In [14]:
import numpy as np

In [15]:
X = np.array(X)
Y = np.array(Y)

In [16]:
X.shape

(841, 224, 224, 3)

In [17]:
Y.shape

(841,)

In [18]:
import cv2

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [20]:
X_train.shape

(672, 224, 224, 3)

In [21]:
X_test.shape

(169, 224, 224, 3)

In [22]:
from keras.applications.vgg16 import VGG16

In [23]:
vgg = VGG16()

In [24]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [25]:
from keras import Sequential

In [26]:
model = Sequential()

In [27]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [29]:
for layer in model.layers:
    layer.trainable=False

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [31]:
from keras.layers import Dense

In [32]:
model.add(Dense(1,activation='sigmoid'))

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [34]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [35]:
model.fit(X_train,Y_train,epochs=1,validation_data=(X_test,Y_test))

21/21 [==============================] - 395s 19s/step - loss: 0.1879 - accuracy: 0.9137 - val_loss: 0.0216 - val_accuracy: 0.9941


In [36]:
cap = cv2.VideoCapture(0)

In [37]:
def detect_face_mask(img):
#     y_pred = model.predict_classes(img.reshape(1,224,224,3))
    y_pred = (model.predict(img.reshape(1,224,224,3)) > 0.5).astype("int32")
    return y_pred[0][0]

In [38]:
sample1 = cv2.imread('sample/1.jpg')
sample1 = cv2.resize(sample1,(224,224))

In [39]:
test1 = detect_face_mask(sample1)
if test1 == 0:
    print("MASK DETECTED")
else:
    print("NO MASK DETECTED")

1/1 [==============================] - 1s 1s/step
MASK DETECTED


In [40]:
sample2 = cv2.imread('sample/2.jpg')
sample2 = cv2.resize(sample2,(224,224))

In [41]:
test2 = detect_face_mask(sample2)
if test2 == 0:
    print("MASK DETECTED")
else:
    print("NO MASK DETECTED")

1/1 [==============================] - 1s 666ms/step
NO MASK DETECTED


In [42]:
def draw_label(img,text,pos,bg_color):
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [43]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [44]:
def detect_face(img):
    coods = haar.detectMultiScale(img)
    return coods

In [48]:
while True:
    ret, frame = cap.read()
    img = cv2.resize(frame,(224,224))
    y_pred = detect_face_mask(img)
    draw_label(frame,"Face mask detection",(30,30),(255,0,0))
    
    #draw face recognition rectangle
    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    for x,y,w,h in coods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,255),5)
        
    # print on jupyter
    if y_pred == 1:
        draw_label(frame,"NO MASK",(450,300),(0,0,255))
        print("NO MASK DETECTED")
    else:
        draw_label(frame,"MASK DETECTED",(380,250),(0,255,0))
        print("NO MASK")
      
    cv2.imshow("window",frame)
    #cv2.resizeWindow("window",400,400)
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cv2.destroyAllWindows()

1/1 [==============================] - 1s 711ms/step
NO MASK DETECTED
1/1 [==============================] - 0s 288ms/step
NO MASK DETECTED
1/1 [==============================] - 0s 279ms/step
NO MASK DETECTED
1/1 [==============================] - 0s 328ms/step
NO MASK DETECTED
1/1 [==============================] - 0s 263ms/step
NO MASK DETECTED
1/1 [==============================] - 0s 306ms/step
NO MASK DETECTED
1/1 [==============================] - 0s 263ms/step
NO MASK
1/1 [==============================] - 0s 276ms/step
NO MASK DETECTED
1/1 [==============================] - 0s 259ms/step
NO MASK
1/1 [==============================] - 0s 264ms/step
NO MASK
1/1 [==============================] - 0s 273ms/step
NO MASK
1/1 [==============================] - 0s 261ms/step
NO MASK DETECTED
1/1 [==============================] - 0s 274ms/step
NO MASK DETECTED
1/1 [==============================] - 0s 270ms/step
NO MASK DETECTED
1/1 [==============================] - 0s 270ms/step
NO 